In [2]:
from typing import List

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

from flask_cors import CORS

import pandas as pd

import os
import requests
import io

In [3]:
url='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master/GUI_in.csv'
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=',')

df['Dprio']=df['Dprio'].astype('float')
df['NumPax']=df['NumPax'].astype('float')
df['Ideal_lower']=df['Ideal_lower'].astype('float')
df['Ideal_average_downweighted']=df['Ideal_average_downweighted'].astype('float')
df['Ideal_average_full']=df['Ideal_average_full'].astype('float')
df['Ideal_upper']=df['Ideal_upper'].astype('float')

dep_dates = df['DepDate'].unique()

In [4]:
df

,DepDate,dtime,Direction,dday,Dprio,NumPax,Ideal_lower,Ideal_average_downweighted,Ideal_average_full,Ideal_upper
0,2019-04-01,960,I,Monday,5.0,22.0,39.0,44.0,62.0,65.0
1,2019-04-01,960,I,Monday,6.0,22.0,37.0,43.0,60.0,64.0
2,2019-04-01,960,I,Monday,7.0,21.0,35.0,42.0,59.0,63.0
3,2019-04-01,960,I,Monday,8.0,20.0,34.0,41.0,57.0,61.0
4,2019-04-01,960,I,Monday,9.0,20.0,32.0,39.0,55.0,60.0
5,2019-04-01,960,I,Monday,10.0,18.0,31.0,38.0,54.0,59.0
6,2019-04-01,960,I,Monday,11.0,20.0,30.0,37.0,52.0,58.0
7,2019-04-01,960,I,Monday,12.0,20.0,28.0,36.0,51.0,56.0
8,2019-04-01,960,I,Monday,13.0,20.0,27.0,35.0,49.0,55.0
9,2019-04-01,960,I,Monday,14.0,19.0,26.0,34.0,48.0,54.0


In [ ]:
COLORS = [
    {
        'background': '#FF0000',
        'text': 'black'
    },
    {
        'background': '#5DFC0A',
        'text': 'black'
    },
]

def cell_style(value1, value2, value3):

    # style = {}
    relative_value1 = value1 - value2
    relative_value2 = value1 - value3

    if relative_value1 < 0 or relative_value2 > 0:
            style = {
                'backgroundColor': COLORS[0]['background'],
                'color': COLORS[0]['text']
            }
    else:
            style = {
                'backgroundColor': COLORS[1]['background'],
                'color': COLORS[1]['text']
            }
    return style

def generate_table(dataframe, max_rows=50):
    rows = []
    for i in range(min(len(dataframe), max_rows)):
        row = []
        for col in dataframe.columns:
            if col == 'NumPax':
                value1 = dataframe.iloc[i][col]
                value2 = dataframe.iloc[i]['Ideal_lower']
                value3 = dataframe.iloc[i]['Ideal_upper']
                style = cell_style(value1, value2, value3)
                row.append(html.Td(value1, style=style))
            else:
                value = dataframe.iloc[i][col]
                row.append(html.Td(value))
        rows.append(html.Tr(row))

    return html.Table(
            # Header
            [html.Tr([html.Th(col) for col in dataframe.columns])] +

            # Body
            rows)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H4(children='Booking curve steering system'),
    dcc.Dropdown(id = 'dropdown',
    options=[{
    'label': i,
    'value': i
    } for i in dep_dates],
    value='All Departure Dates'),
    # generate_table(df)
    html.Div(id='table-container'),
])

@app.callback(Output('table-container', 'children'), [Input('dropdown', 'value')])
def update_table(value):
    dff = df[df['DepDate'] == value] # update with your own logic
    return generate_table(dff)

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Apr/2019 10:04:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 10:04:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 10:04:00] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 10:04:00] "GET /_favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 10:04:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 10:04:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 10:07:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 10:07:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 10:07:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 10:08:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 10:08:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 10:09:00] "POST /_dash-update-component HTTP/1.1" 